In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns

# === Device Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load SEED EEG Features ===
DATA_PATH = "D:/SEED_IV/SEED_IV/eeg_feature_smooth"

def pad_or_truncate(features, target_length):
    current_length = len(features)
    if current_length < target_length:
        return np.pad(features, (0, target_length - current_length), mode='constant')
    else:
        return features[:target_length]

def load_eeg_features(sessions=["1", "2", "3"], feature_type="de_movingAve", num_channels=62):
    data = []
    labels = []
    max_length = 0

    for session in sessions:
        session_path = os.path.join(DATA_PATH, session)
        if not os.path.exists(session_path):
            continue

        for file in os.listdir(session_path):
            if not file.endswith(".mat"):
                continue

            file_path = os.path.join(session_path, file)
            mat = scipy.io.loadmat(file_path)

            for trial in range(1, 25):
                key = f"{feature_type}{trial}"
                if key not in mat:
                    continue

                feature_matrix = mat[key]
                if feature_matrix.shape[0] != num_channels:
                    continue

                trial_features = feature_matrix.reshape(-1)
                max_length = max(max_length, len(trial_features))
                data.append(trial_features)
                labels.append((trial - 1) % 3)

    if not data:
        return np.array([]), np.array([])

    data_fixed = np.array([pad_or_truncate(x, max_length) for x in data])
    return np.array(data_fixed), np.array(labels)

# === Load & Normalize Data ===
X_fixed, y = load_eeg_features()
scaler = StandardScaler()
X_fixed = scaler.fit_transform(X_fixed)

X_tensor = torch.tensor(X_fixed, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

# === CBAM Model ===
class CBAM(nn.Module):
    def __init__(self, in_channels):
        super(CBAM, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 2, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels // 2, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel attention
        channel_attention = self.channel_attention(x)
        x = x * channel_attention
        
        # Spatial attention
        spatial_attention = self.spatial_attention(x)
        x = x * spatial_attention
        
        return x

class EEGNet_CBAM(nn.Module):
    def __init__(self, num_classes=3, input_channels=62, input_samples=320):
        super(EEGNet_CBAM, self).__init__()

        # Temporal convolution
        self.temporal_conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 64), padding=(0, 32), bias=False),
            nn.BatchNorm2d(16),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        # Depthwise convolution
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(input_channels, 1), groups=16, bias=False),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        # Separable convolution
        self.separable_conv = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(1, 16), padding=(0, 8), bias=False),
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        # CBAM module
        self.cbam = CBAM(in_channels=64)

        # Calculate the output size after the convolution layers
        self._to_linear = None
        self.convs = nn.Sequential(
            self.temporal_conv,
            self.depthwise_conv,
            self.separable_conv,
            self.cbam
        )
        self._calc_linear_size(input_samples)

        # Fully connected layer
        self.fc1 = nn.Linear(self._to_linear, 128)
        self.fc2 = nn.Linear(128, num_classes)

        self.num_classes = num_classes
        self.input_samples = input_samples

    def _calc_linear_size(self, input_samples):
        # Create a dummy input to calculate the output size after convolutions
        with torch.no_grad():
            dummy_input = torch.ones(1, 1, 62, input_samples)
            x = self.convs(dummy_input)
            self._to_linear = int(np.prod(x.size()))

    def forward(self, x):
        # Pass through the convolution layers
        x = self.convs(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = self.fc2(x)

        return x

# === Model Initialization ===
model = EEGNet_CBAM(num_classes=3, input_channels=62, input_samples=320).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

# === Training Loop ===
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        batch_X = batch_X.view(-1, 1, 62, 320)  # Reshape for the model

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

print("✅ EEGNet+CBAM Training Complete!")

# === Evaluation ===
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        batch_X = batch_X.view(-1, 1, 62, 320)  # Reshape for the model

        outputs = model(batch_X)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())

# Metrics
acc = accuracy_score(all_targets, all_preds)
report = classification_report(all_targets, all_preds, digits=4)
print(f"\nAccuracy: {acc:.4f}")
print("\nClassification Report:\n", report)


Epoch [1/50], Loss: 47.4409
Epoch [2/50], Loss: 31.6503
Epoch [3/50], Loss: 25.0713
Epoch [4/50], Loss: 19.8801
Epoch [5/50], Loss: 18.4690
Epoch [6/50], Loss: 15.2755
Epoch [7/50], Loss: 13.0785
Epoch [8/50], Loss: 12.3072
Epoch [9/50], Loss: 10.9445
Epoch [10/50], Loss: 9.6125
Epoch [11/50], Loss: 7.8980
Epoch [12/50], Loss: 6.2396
Epoch [13/50], Loss: 6.3804
Epoch [14/50], Loss: 5.3582
Epoch [15/50], Loss: 4.0992
Epoch [16/50], Loss: 3.5946
Epoch [17/50], Loss: 3.0887
Epoch [18/50], Loss: 3.0847
Epoch [19/50], Loss: 3.7791
Epoch [20/50], Loss: 3.8239
Epoch [21/50], Loss: 3.8036
Epoch [22/50], Loss: 2.1579
Epoch [23/50], Loss: 1.3307
Epoch [24/50], Loss: 1.6894
Epoch [25/50], Loss: 1.3495
Epoch [26/50], Loss: 1.0611
Epoch [27/50], Loss: 1.2763
Epoch [28/50], Loss: 0.7897
Epoch [29/50], Loss: 0.7817
Epoch [30/50], Loss: 0.8607
Epoch [31/50], Loss: 0.9403
Epoch [32/50], Loss: 0.7058
Epoch [33/50], Loss: 0.4972
Epoch [34/50], Loss: 0.8531
Epoch [35/50], Loss: 0.5045
Epoch [36/50], Loss:

In [3]:
def region_wise_analysis_cbam(model, X_tensor, y_tensor, region_channels_dict, total_channels=62, input_samples=320):
    model.eval()
    results = {}

    for region, channels in region_channels_dict.items():
        X_region = X_tensor[:, channels, :]  # Extract region-specific channels

        # Pad to 62 channels with zeros for unused channels
        padded = torch.zeros((X_region.shape[0], total_channels, input_samples))
        padded[:, channels, :] = X_region

        # Reshape for model input
        padded = padded.unsqueeze(1).to(device)  # Shape: [N, 1, 62, 320]
        y_tensor_device = y_tensor.to(device)

        # Evaluate region-specific input
        with torch.no_grad():
            outputs = model(padded)
            preds = torch.argmax(outputs, dim=1)
            acc = accuracy_score(y_tensor_device.cpu().numpy(), preds.cpu().numpy())

        results[region] = acc

    return results

# === Define Region Mapping (adjust indices as per EEG cap layout if needed) ===
region_channels = {
    'Frontal': list(range(0, 20)),      # Approximate
    'Temporal': list(range(20, 30)),
    'Parietal': list(range(30, 45)),
    'Occipital': list(range(45, 52)),
    'Central': list(range(52, 62))
}

# === Run Region-wise Analysis ===
region_scores_cbam = region_wise_analysis_cbam(model, X_tensor.view(-1, 62, 320), y_tensor, region_channels)

print("\n🧠 Region-wise Accuracy (CBAM):")
for region, score in region_scores_cbam.items():
    print(f"{region}: {score:.4f}")



🧠 Region-wise Accuracy (CBAM):
Frontal: 0.8009
Temporal: 0.5676
Parietal: 0.5972
Occipital: 0.3880
Central: 0.3806


In [5]:
from torchinfo import summary

# EEGNet_CBAM expects input shape: [batch, 1, 62, 320]
summary(model, input_size=(16, 1, 62, 320))


Layer (type:depth-idx)                   Output Shape              Param #
EEGNet_CBAM                              [16, 3]                   --
├─Sequential: 1-1                        [16, 64, 1, 322]          --
│    └─Sequential: 2-1                   [16, 16, 62, 321]         --
│    │    └─Conv2d: 3-1                  [16, 16, 62, 321]         1,024
│    │    └─BatchNorm2d: 3-2             [16, 16, 62, 321]         32
│    │    └─ELU: 3-3                     [16, 16, 62, 321]         --
│    │    └─Dropout: 3-4                 [16, 16, 62, 321]         --
│    └─Sequential: 2-2                   [16, 32, 1, 321]          --
│    │    └─Conv2d: 3-5                  [16, 32, 1, 321]          1,984
│    │    └─BatchNorm2d: 3-6             [16, 32, 1, 321]          64
│    │    └─ELU: 3-7                     [16, 32, 1, 321]          --
│    │    └─Dropout: 3-8                 [16, 32, 1, 321]          --
│    └─Sequential: 2-3                   [16, 64, 1, 322]          --
│    │   